In [1]:
import anndata
import torch
import stPlus

import squidpy as sq
import numpy as np
import scanpy as sc
import pandas as pd

from sklearn.model_selection import KFold
from transpa.eval_util import calc_corr
from transpa.util import expTransImp, leiden_cluster, compute_autocorr
from benchmark import SpaGE_impute, Tangram_impute
import warnings

warnings.filterwarnings('ignore')

seed = 10
device = torch.device("cuda:2") if torch.cuda.is_available() else torch.device("cpu")

In [2]:
spatial_df_file = '../../data/ST/osmFISH/osmFISH_df.csv'
spatial_loom_file = '../../data/ST/osmFISH/osmFISH_SScortex_mouse_all_cells.loom'

VISp_adata = sc.read("../../data/scRNAseq/Allen_VISp/mouse_VISp_2018-06-14_exon-matrix.csv").T
genes = pd.read_csv("../../data/scRNAseq/Allen_VISp/mouse_VISp_2018-06-14_genes-rows.csv", header=0,sep=',')
VISp_meta = pd.read_csv("../../data/scRNAseq/Allen_VISp/mouse_VISp_2018-06-14_samples-columns.csv", header=0,sep=',')

VISp_adata.obs = VISp_meta
VISp_adata.var_names = genes.gene_symbol

sc.pp.filter_genes(VISp_adata, min_cells=10)
VISp_adata = VISp_adata[(VISp_adata.obs['class'] != 'No Class') & (VISp_adata.obs['class'] != 'Low Quality')]
classes, ct_list = leiden_cluster(VISp_adata)
cls_key = 'leiden'
VISp_adata.obs[cls_key] = classes
sc.pp.normalize_total(VISp_adata)
sc.pp.log1p(VISp_adata)
VISp_adata


AnnData object with n_obs × n_vars = 14249 × 34617
    obs: 'sample_name', 'sample_id', 'sample_type', 'organism', 'donor', 'sex', 'age_days', 'eye_condition', 'genotype', 'driver_lines', 'reporter_lines', 'brain_hemisphere', 'brain_region', 'brain_subregion', 'injection_label_direction', 'injection_primary', 'injection_secondary', 'injection_tract', 'injection_material', 'injection_exclusion_criterion', 'facs_date', 'facs_container', 'facs_sort_criteria', 'rna_amplification_set', 'library_prep_set', 'library_prep_avg_size_bp', 'seq_name', 'seq_tube', 'seq_batch', 'total_reads', 'percent_exon_reads', 'percent_intron_reads', 'percent_intergenic_reads', 'percent_rrna_reads', 'percent_mt_exon_reads', 'percent_reads_unique', 'percent_synth_reads', 'percent_ecoli_reads', 'percent_aligned_reads_total', 'complexity_cg', 'genes_detected_cpm_criterion', 'genes_detected_fpkm_criterion', 'tdt_cpm', 'gfp_cpm', 'class', 'subclass', 'cluster', 'confusion_score', 'cluster_correlation', 'core_intermed

In [3]:
osmFISH = sc.read_loom(spatial_loom_file)
osmFISH = osmFISH[~np.isin(osmFISH.obs.Region, ['Excluded', 'Hippocampus', 'Internal Capsule Caudoputamen','Ventricle', 'White matter'])].copy()
raw_spatial_df  = pd.read_csv(spatial_df_file)
osmFISH.X = raw_spatial_df.values


raw_scrna_df    = pd.DataFrame(VISp_adata.X, columns=VISp_adata.var_names)
adata_scrna   = VISp_adata
raw_spatial_df.to_csv('../../output/osmFISH_raw.csv')

raw_shared_gene = np.intersect1d(raw_spatial_df.columns, raw_scrna_df.columns)
raw_spatial_df.shape, raw_scrna_df.shape, raw_shared_gene.shape

((3405, 33), (14249, 34617), (33,))

In [4]:
osmFISH.obsm['spatial'] = np.hstack([osmFISH.obs.X.values.reshape(-1,1), osmFISH.obs.Y.values.reshape(-1,1)])
np.save('../../output/osmFISH_locations.npy', osmFISH.obsm['spatial'])
sq.gr.spatial_neighbors(osmFISH)

In [5]:
kf = KFold(n_splits=5, shuffle=True, random_state=0)
kf.get_n_splits(raw_shared_gene)

df_transImpSpa = pd.DataFrame(np.zeros((osmFISH.n_obs, len(raw_shared_gene))), columns=raw_shared_gene)
df_transImpCls = pd.DataFrame(np.zeros((osmFISH.n_obs, len(raw_shared_gene))), columns=raw_shared_gene)
df_transImpClsSpa = pd.DataFrame(np.zeros((osmFISH.n_obs, len(raw_shared_gene))), columns=raw_shared_gene)
df_transImp = pd.DataFrame(np.zeros((osmFISH.n_obs, len(raw_shared_gene))), columns=raw_shared_gene)
df_stplus_res = pd.DataFrame(np.zeros((osmFISH.n_obs, len(raw_shared_gene))), columns=raw_shared_gene)
df_spaGE_res = pd.DataFrame(np.zeros((osmFISH.n_obs, len(raw_shared_gene))), columns=raw_shared_gene)
df_tangram_res = pd.DataFrame(np.zeros((osmFISH.n_obs, len(raw_shared_gene))), columns=raw_shared_gene)


for idx, (train_ind, test_ind) in enumerate(kf.split(raw_shared_gene)):    
    print(f"\n===== Fold {idx+1} =====\nNumber of train genes: {len(train_ind)}, Number of test genes: {len(test_ind)}")
    train_gene = raw_shared_gene[train_ind]
    test_gene  = raw_shared_gene[test_ind]
    
    test_spatial_df = raw_spatial_df[test_gene]
    spatial_df = raw_spatial_df[train_gene]
    scrna_df   = raw_scrna_df

    df_transImpSpa[test_gene] = expTransImp(
            df_ref=raw_scrna_df,
            df_tgt=raw_spatial_df,
            train_gene=train_gene,
            test_gene=test_gene,
            signature_mode='cell',
            mapping_mode='lowrank',
            spa_adj=osmFISH.obsp['spatial_connectivities'].tocoo(),
            seed=seed,
            device=device)

    corr_transImpSpa_res = calc_corr(raw_spatial_df, df_transImpSpa, test_gene)
    print(f'fold {idx}, median correlation: {np.median(corr_transImpSpa_res)} (TransImpSpa)')

    df_transImpCls[test_gene] = expTransImp(
            df_ref=raw_scrna_df,
            df_tgt=raw_spatial_df,
            train_gene=train_gene,
            test_gene=test_gene,
            ct_list=ct_list,
            classes=classes,
            signature_mode='cluster',
            mapping_mode='full',
            seed=seed,
            device=device)

    corr_transImpSpa_res = calc_corr(raw_spatial_df, df_transImpCls, test_gene)
    print(f'fold {idx}, median correlation: {np.median(corr_transImpSpa_res)} (TransImpCls)')

    df_transImp[test_gene] = expTransImp(
            df_ref=raw_scrna_df,
            df_tgt=raw_spatial_df,
            train_gene=train_gene,
            test_gene=test_gene,
            signature_mode='cell',
            mapping_mode='lowrank',
            seed=seed,
            device=device)

    corr_transImpSpa_res = calc_corr(raw_spatial_df, df_transImp, test_gene)
    print(f'fold {idx}, median correlation: {np.median(corr_transImpSpa_res)} (TransImp)')

    df_transImpClsSpa[test_gene] = expTransImp(
            df_ref=raw_scrna_df,
            df_tgt=raw_spatial_df,
            train_gene=train_gene,
            test_gene=test_gene,
            ct_list=ct_list,
            classes=classes,
            spa_adj=osmFISH.obsp['spatial_connectivities'].tocoo(),
            signature_mode='cluster',
            mapping_mode='full',
            wt_spa=1.0,
            seed=seed,
            device=device)

    corr_transImpSpa_res = calc_corr(raw_spatial_df, df_transImpClsSpa, test_gene)
    print(f'fold {idx}, median correlation: {np.median(corr_transImpSpa_res)} (TransImpClsSpa)')

    df_stplus_res[test_gene] = stPlus.stPlus(spatial_df, scrna_df, test_gene, "tmp_osmvisp", verbose=False, random_seed=seed, device=device)
    corr_res_stplus = calc_corr(raw_spatial_df, df_stplus_res, test_gene)
    print(f'\t\t\t{np.median(corr_res_stplus)} (stPlus)')

    df_spaGE_res[test_gene]  = SpaGE_impute(scrna_df, spatial_df, train_gene, test_gene)
    corr_res_spaGE = calc_corr(raw_spatial_df, df_spaGE_res, test_gene)
    print(f'\t\t\t{np.median(corr_res_spaGE)} (spaGE)')

    df_tangram_res[test_gene] = Tangram_impute(adata_scrna, osmFISH, train_gene, test_gene, device, cls_key)
    corr_res_tangram = calc_corr(raw_spatial_df, df_tangram_res, test_gene)
    print(f'\t\t\t{np.median(corr_res_tangram)} (Tangram)')

corr_transImpSpa_res = calc_corr(raw_spatial_df, df_transImpSpa, raw_shared_gene)
corr_transImp_res = calc_corr(raw_spatial_df, df_transImp, raw_shared_gene)
corr_transImpCls_res = calc_corr(raw_spatial_df, df_transImpCls, raw_shared_gene)
corr_transImpClsSpa_res = calc_corr(raw_spatial_df, df_transImpClsSpa, raw_shared_gene)
corr_res_stplus = calc_corr(raw_spatial_df, df_stplus_res, raw_shared_gene)
corr_res_spaGE = calc_corr(raw_spatial_df, df_spaGE_res, raw_shared_gene)
corr_res_tangram = calc_corr(raw_spatial_df, df_tangram_res, raw_shared_gene)   

print(np.median(corr_transImpSpa_res), "(TransImpSpa)", 
      np.median(corr_transImp_res), "(TransImp)", 
      np.median(corr_transImpCls_res), "(TransImpCls)", 
      np.median(corr_transImpClsSpa_res), "(TransImpClsSpa)", 
      np.median(corr_res_stplus), "(stPlus)", 
      np.median(corr_res_spaGE), "(spaGE)",
      np.median(corr_res_tangram), "(Tangram)"
      )


===== Fold 1 =====
Number of train genes: 26, Number of test genes: 7


[TransImp] Epoch: 1000/1000, loss: 0.138717, (IMP) 0.138639, (SPA) 1.0 x 0.000078: 100%|██████████| 1000/1000 [00:09<00:00, 109.71it/s]


fold 0, median correlation: 0.24938531741895364 (TransImpSpa)


[TransImp] Epoch: 1000/1000, loss: 0.223927, (IMP) 0.223927, (SPA) 1.0 x 0.000000: 100%|██████████| 1000/1000 [00:03<00:00, 252.42it/s]


fold 0, median correlation: 0.21658029062353512 (TransImpCls)


...

fold 4, median correlation: 0.16437409527506697 (TransImpClsSpa)
			0.12186984030028128 (stPlus)
			0.1365013643542471 (spaGE)


INFO:root:27 training genes are saved in `uns``training_genes` of both single cell and spatial Anndatas.
INFO:root:27 overlapped genes are saved in `uns``overlap_genes` of both single cell and spatial Anndatas.
INFO:root:uniform based density prior is calculated and saved in `obs``uniform_density` of the spatial Anndata.
INFO:root:rna count based density prior is calculated and saved in `obs``rna_count_based_density` of the spatial Anndata.
INFO:root:Allocate tensors for mapping.
INFO:root:Begin training with 27 genes and rna_count_based density_prior in clusters mode...
INFO:root:Printing scores every 100 epochs.


Score: 0.648, KL reg: 0.066
Score: 0.898, KL reg: 0.001
Score: 0.900, KL reg: 0.001
Score: 0.901, KL reg: 0.001
Score: 0.901, KL reg: 0.001
Score: 0.901, KL reg: 0.001
Score: 0.901, KL reg: 0.001
Score: 0.901, KL reg: 0.001
Score: 0.901, KL reg: 0.001
Score: 0.901, KL reg: 0.001


INFO:root:Saving results..


			0.13492312622199132 (Tangram)
0.24938531741895364 (TransImpSpa) 0.24516623725980274 (TransImp) 0.2065641754955636 (TransImpCls) 0.2035653050176823 (TransImpClsSpa) 0.18881567157568477 (stPlus) 0.17948876454543866 (spaGE) 0.19108899903281826 (Tangram)


In [6]:
df_transImp.to_csv('../../output/osmFISH_AllenVISp_transImpute.csv')
df_transImpSpa.to_csv('../../output/osmFISH_AllenVISp_transImpSpa.csv')
df_transImpCls.to_csv('../../output/osmFISH_AllenVISp_transImpCls.csv')
df_transImpClsSpa.to_csv('../../output/osmFISH_AllenVISp_transImpClsSpa.csv')
df_spaGE_res.to_csv('../../output/osmFISH_AllenVISp_spaGE.csv')
df_stplus_res.to_csv('../../output/osmFISH_AllenVISp_stPlus.csv')
df_tangram_res.to_csv('../../output/osmFISH_AllenVISp_Tangram.csv')

In [7]:
dict_df = {"TransImp":df_transImp, 
           "TransImpSpa":df_transImpSpa, 
           "TransImpCls":df_transImpCls,
           "TransImpClsSpa":df_transImpClsSpa,
           "spaGE": df_spaGE_res, "stPlus": df_stplus_res, 
           "Tangram":df_tangram_res
           }
sq.gr.spatial_autocorr(
    osmFISH,
    n_jobs=10,
)
sq.gr.spatial_autocorr(
    osmFISH,
    n_jobs=10,
    mode='geary',
)

dict_adata = {name: compute_autocorr(osmFISH[:, raw_shared_gene].copy(), df) for name, df in dict_df.items()}


In [8]:
from sklearn.metrics import mean_squared_error
moranIs = {name:mean_squared_error(osmFISH.uns['moranI'].loc[raw_shared_gene].I, imp_adata.uns['moranI'].loc[raw_shared_gene].I) for name, imp_adata in dict_adata.items()}
gearyCs = {name:mean_squared_error(osmFISH.uns['gearyC'].loc[raw_shared_gene].C, imp_adata.uns['gearyC'].loc[raw_shared_gene].C) for name, imp_adata in dict_adata.items()}

print("Mean Squared Error\nMoran's I:\n")
print("\n".join([f"\tTrue vs {method}: {score:.6f}" for method, score in moranIs.items()]))
print("Geary's C:\n")
print("\n".join([f"\tTrue vs {method}: {score:.6f}" for method, score in gearyCs.items()]))


Mean Squared Error
Moran's I:

	True vs TransImp: 0.016484
	True vs TransImpSpa: 0.015250
	True vs TransImpCls: 0.024275
	True vs TransImpClsSpa: 0.023119
	True vs spaGE: 0.023299
	True vs stPlus: 0.025360
	True vs Tangram: 0.032321
Geary's C:

	True vs TransImp: 0.016447
	True vs TransImpSpa: 0.015162
	True vs TransImpCls: 0.024567
	True vs TransImpClsSpa: 0.023436
	True vs spaGE: 0.024581
	True vs stPlus: 0.025837
	True vs Tangram: 0.032471
